# Configure (or Reconfigure) MintPy Time Series Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

The configuration for a MintPy Time Series Analysis is defined in a config file called [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/main/src/mintpy/defaults/smallbaselineApp.cfg).

This file contains all optional parameters with their default settings.

When using MintPy, do not edit this file directly. Instead, you will create a custom config file that contains only the parameters that you wish to update. MintPy will load the custom config, using it to overwrite default settings in `smallbaselineApp.cfg`.

**You must reprocess the time series after making configuration updates**

You can update the configuration without needing to reload the source input data. However, you must rerun the `modify_network` step to register the updated configuration, and then rerun the remaining [smallbaselineApp.py](https://github.com/insarlab/MintPy/blob/main/src/mintpy/smallbaselineApp.py) steps to reprocess the time series with your updates.

If you make updates in this notebook, skip reprocessing the time series, and jump directly to re-plotting or outputting to GeoTiff, you will see results from the previous time series, not from your most recent updates. 

---
## 0. Import Required Software

In [ ]:
from ipyfilechooser import FileChooser
from pathlib import Path
from pprint import pprint
import re
import mintpy

import ipywidgets as widgets

---
# 1. Examine Current State of Custom Config File

**Select your custom config file**

- It will be located in your loaded MintPy time series' `MintPy` directory
- It is a text file with your MintPy project name
  - ie., `path/to/MintPy/my_project.txt`

In [ ]:
path = Path.home()
fc = FileChooser(path)
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

**Open and display the current contents of the custom config**

In [ ]:
custom_config_dict = {}

custom_config_path = Path(fc.selected)
with open(custom_config_path, 'r') as f:
    custom_lines = f.readlines()

custom_lines = [l for l in custom_lines if not l.startswith('\n') and not l.strip().startswith('#')]

for l in custom_lines:
    param = l.split(' ')[0]
    info = l.split('= ')[-1].strip()
    custom_config_dict[param] = info

print('Current custom config:\n')
for l in custom_lines:
    print(l)

In [ ]:
custom_config_dict

---
## 2. Update the Current Configuration 

**Read in the default config**

In [ ]:
default_config_path = Path(mintpy.__file__).parent / "defaults/smallbaselineApp.cfg"
with open(default_config_path, 'r') as f:
    lines = f.readlines()
    
lines = [l for l in lines if not l.startswith('\n')]

**Complete an interactive config form**

***Only complete the fields you wish to change***

- The form contains settings from your existing custom config and parameter hints for any unused attributes
- Uncompleted fields will be set to 'auto'

**Some Commonly Updated Options:**

- Set a reference point
  -  `mintpy.reference.lalo`
  -  `mintpy.reference.yx`
- Use the time series start date as the reference date
  - `nmintpy.reference.date = no`
- Use multihreaded processing for inversion and DEM correction
  - `mintpy.compute.cluster = local`
  - `mintpy.compute.numWorker`
- Subset
  - `mintpy.subset.lalo`
  - `mintpy.subset.yx`

In [ ]:
param_dict = {}

layout = widgets.Layout(width='initial', height='40px') #set width and height

for l in lines:
    if l.startswith('#'):
        print(l)
    else:
        param = l.split(' ')[0]
        if param in custom_config_dict.keys():
            info = custom_config_dict[param]
        else:
            info = l.split('= auto ')[-1].strip()[1:]  
        param_dict[param] =  widgets.Text(
            placeholder=info,
            description=f'{param}:',
            disabled=False,
            align_items='stretch', 
            layout = layout,
            style= {'description_width': 'initial'},
        )
        display(param_dict[param])

**Create updated list of custom configuration settings**

- Check your settings before overwriting your config file
- Any settings not included in your custom config will be treated as `auto` by MintPy

In [ ]:
updated_config = ""

for k, v in param_dict.items():
    if len(v.value) > 0 and v.value != v.placeholder:
        updated_config += f'{k}: {v.value}\n'
    elif k in custom_config_dict.keys() and v.placeholder == custom_config_dict[k]:
        updated_config += f'{k}: {v.placeholder}\n'
print('Updated custom config settings:\n')
pprint(updated_config)

**Overwrite your custom config with updated settings**

- If you are happy with the output from the previous cell, overwrite your config file

In [ ]:
custom_config_path.write_text(updated_config)

**It is now time to run the `4_MintPy_Time_Series.ipynb` (Perform MintPy Time Series Analysis) Notebook**

If you skip rerunning the time series and create plots or Geotiff output now, the changes you just made will not be applied. 